In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env
from itertools import combinations
from time import sleep
from IPython.display import Image,display, clear_output

In [2]:



def dfcheck(data):
    df=data
    print(f'info:\n{df.info()}\n\n')   
    print(f'describe:\n{  df.describe()}\n\n')
    print(f'shape:\n{df.shape}\n\n')
    print(f'head:\n')
    display(df.head(3))
    
   
def compareAgivenBvsBgivenA(df_l,df_r,a,b,playbyplay=False):


    #here I sort each data fram to make a one to one comparison  I even reindex
    df_l=df_l.sort_values(by=[a,b])
    df_r=df_r.sort_values(by=[a,b])
    cols=df_l.columns.to_list()
    df_r=df_r[cols]
    df_r=df_r.reset_index(drop=True)
# now I show the two  note the different lengths
    
    comp=pd.merge(left=df_l,right=df_r,how='inner',on=[a,b])


    compcols=comp.columns.to_list()
    compcols.sort()
    comp=comp[compcols]
    compcols=comp.columns.to_list()
    comparecolset=set(compcols)-{'ip','user_agent'}
    compcols=list(comparecolset)
    compcols.sort()
    # print(compcols)
    #uncomment for troubleshooting if needed



    #should comp cols always be even with this logic and it wil be since you always add 2 dfs
    boollist=[]

  

    

    for i in range(1,len(compcols)):
        if (i%2==1)and(playbyplay==True):
            col_a=compcols[i-1]
            col_b=compcols[i]
            b=comp[col_b]
            a=comp[col_a]
            print(f' Aggregate sum of the percent difference of:\n {col_a} v {col_b} \n With decimial precision of e-8\n')
            listofdiff=(round(((a-b))/len(a)*100,8))
            sumofdiff=sum(list(listofdiff))
            print(f'{sumofdiff}')
            print('\n\n')
            sleep(2.5)
            clear_output()
            sleep(2.5)

        elif (i%2==1)and(playbyplay==False) :
            col_a=compcols[i-1]
            col_b=compcols[i]
            b=comp[col_b]
            a=comp[col_a]
        
            listofdiff=(round(((a-b))/len(a)*100,8))
            sumofdiff=sum(list(listofdiff))
            boollist.append(sumofdiff!=0)
            
        else:
            continue
    if len(boollist)>0:
        return boollist
    else:
        return None




def categoricals(data,remove=[],catncut=2):
    '''
    Uses logic to make a list of all the categoical cols
    
    '''
    remove=set(remove)
    df=data
    catset=df.select_dtypes(exclude='number').nunique().sort_values(ascending=False).index.to_list()
    catset=list(set(catset)-remove)
    catsetadd=(df.select_dtypes(include='number').nunique())
    catsetadd=catsetadd[(catsetadd<2)].to_list()
    if len(catsetadd)>0:
        catset.extend(catsetadd)
    # print()
    # print(catset)
    return catset
    

   


def catcombos(data,n,remove=['ip'],catncut=0):
    '''
    n must be an int
    this gives you n length tuples of categorical col names in a list that is sorted from greates number to least
   
    
    
    
    '''
   
    df=data
    catset=categoricals(df,remove=remove,catncut=catncut)

    # print(catset)
    combolist=(combinations(catset,n))
    combos=list(combolist);combos
    combos=combos[0];combos
    combos=[combos[i] for i in range(len(combos))];combos

    return combos







    
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)





url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
# df = pd.concat([df.entry, df.entry.apply(parse_log_entry)], axis=1)








def empricalprobdictionary(data,remove=[],catncut=2):
    df=data
    catset=categoricals(df,remove=remove,catncut=catncut)
    empprobdict={}
    for c in catset:
       
        empprobdict.update(pd.DataFrame(df[c].value_counts(normalize=True)).to_dict())
    return empprobdict


In [3]:
df = df.entry.apply(parse_log_entry)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13974 entries, 0 to 13973
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ip              13974 non-null  object
 1   timestamp       13974 non-null  object
 2   request_method  13974 non-null  object
 3   request_path    13974 non-null  object
 4   http_version    13974 non-null  object
 5   status_code     13974 non-null  object
 6   size            13974 non-null  int64 
 7   user_agent      13974 non-null  object
dtypes: int64(1), object(7)
memory usage: 873.5+ KB


In [4]:



new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

df = pd.concat([df,new],axis=0,join='outer',ignore_index=True)

dfcheck(df)





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13978 entries, 0 to 13977
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ip              13978 non-null  object
 1   timestamp       13978 non-null  object
 2   request_method  13978 non-null  object
 3   request_path    13978 non-null  object
 4   http_version    13978 non-null  object
 5   status_code     13978 non-null  object
 6   size            13978 non-null  int64 
 7   user_agent      13978 non-null  object
dtypes: int64(1), object(7)
memory usage: 873.8+ KB
info:
None


describe:
               size
count  1.397800e+04
mean   4.500019e+05
std    1.614915e+05
min    0.000000e+00
25%    5.006370e+05
50%    5.101380e+05
75%    5.112910e+05
max    2.056327e+06


shape:
(13978, 8)


head:



,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0


In [5]:



df['size_mb'] = df['size'] / (2**20)
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp',append=False)
dfcheck(df)


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13978 entries, 2019-04-16 19:34:42 to 2019-04-19 19:42:41
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ip              13978 non-null  object 
 1   request_method  13978 non-null  object 
 2   request_path    13978 non-null  object 
 3   http_version    13978 non-null  object 
 4   status_code     13978 non-null  object 
 5   size            13978 non-null  int64  
 6   user_agent      13978 non-null  object 
 7   size_mb         13978 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 982.8+ KB
info:
None


describe:
               size       size_mb
count  1.397800e+04  13978.000000
mean   4.500019e+05      0.429155
std    1.614915e+05      0.154010
min    0.000000e+00      0.000000
25%    5.006370e+05      0.477445
50%    5.101380e+05      0.486506
75%    5.112910e+05      0.487605
max    2.056327e+06      1.961066


shape:
(1397

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472


In [6]:

# (catsetadd.values<2)

# twocats=catcombos(df,2,remove=['ip'],catncut='')

In [7]:


def bigcompare(df,playbyplay=False):
    nm='ip'
    empprobdict=empricalprobdictionary(df,remove=[],catncut=2)
    cats=categoricals(df,remove=[nm],catncut=2)
    # print(cats)
    res=[]
    agivenb={}
    for i in range(len(cats)):
        cat=cats[i]

        a=cat
      

        s1=df.groupby(nm)[a].value_counts(normalize=True).rename(f'P({a}|{nm})').reset_index()
        s1[f'P({nm})']=s1[nm].map(lambda x :empprobdict[nm].get(x))
        s1[f'P({a})']=s1[a].map(lambda x :empprobdict[a].get(x))
        s1[f'P({nm}|{a})']=s1[f'P({a}|{nm})']*s1[f'P({nm})']/s1[f'P({a})']

        s2=df.groupby(a)[nm].value_counts(normalize=True).rename(f'P({nm}|{a})').reset_index()
        s2[f'P({nm})']=s2[nm].map(lambda x :empprobdict[nm].get(x))
        s2[f'P({a})']=s2[a].map(lambda x :empprobdict[a].get(x))
        s2[f'P({a}|{nm})']=s2[f'P({nm}|{a})']*s2[f'P({a})']/s2[f'P({nm})']

        # display(s1,s2)


        res1=compareAgivenBvsBgivenA(df_l=s1,df_r=s2,a=nm,b=a,playbyplay=playbyplay)
        res2=compareAgivenBvsBgivenA(df_l=s2,df_r=s1,a=nm,b=a,playbyplay=playbyplay)
        res.extend(res1)
        res.extend(res2)
        s1=s1.drop(columns=[f'P({a})',f'P({nm})'])
        s1[f'({nm},{a})']=tuple(zip(s1[nm],s1[a]))
        s1=s1.drop(columns=[a,nm])
        cols = list(s1.columns.values)
        cols.reverse()     
        s1=s1[cols]
        s1=s1.set_index([f'({nm},{a})'])
     
        s1=pd.DataFrame(s1).to_dict()
        agivenb.update(s1)
      
    print(f'For all of our categorical data we have a total of \n{sum(res)} instances \nwhere P(A|B) can not be used to solve for P(B|A) at the elementwise level')
    empprobdict.update(agivenb)
    return empprobdict

    








## The point here is to show we can go either way when and if we need to. All this was to show that the LHS=RHS


## This concept is actually intersting because these facts now alow us to ask other questions. Namely we can us the definition of independence to consider if one categorical variable is dependt on the other. This might be useful as a metetric for feature selection. Pehaps this is what is already being done 




In [8]:
empprobdict=bigcompare(df,playbyplay=False)


For all of our categorical data we have a total of 
0 instances 
where P(A|B) can not be used to solve for P(B|A) at the elementwise level


In [64]:

def groupbyexplore(df,grp,slp=2):
    # display(pd.DataFrame((grp.size())))
    # keylist=list(grp.groups.keys())
    groupd={}
    for name, group in grp:
        # print(f'name:\n{name}')
        # print()
        new=pd.DataFrame(group)
        # print(f'shape:\n{new.shape}')
        # print()

        # print(f'size:\n{new.size}')
        groupd.update({f'n{name}':{f'n{name}_length':len(group),f'df_{name}':new}})
       
       

      
        # print('\n\n\n\n')


    # for k in keylist:
    #     x=grp.get_group(k)
    #     display(x.value_counts())
    #     sleep(slp)
    #     clear_output(wait=True)
    return groupd
     



In [65]:
def bigGroupExplore(df,n=5):
    ndict={}    
    combos=catcombos(df,n=n,remove=['ip'],catncut=0)
    ndict.update({f'combinations_{n}':combos})
    
    gr1=df.groupby('ip')[combos]
    
    combos.reverse()
    
    gr2=df.groupby('ip')[combos]


    
    groupd=groupbyexplore(df,gr1)
    ndict[f'combinations_{n}']=groupd
    return ndict
    


    
    # groupbyexplore(df,gr2)







In [66]:
giantdict={}

for i in range(0,6):
    giantdict.update(bigGroupExplore(df,n=i))



In [71]:

len(giantdict)


6

In [1]:

     

# for c in twocats:
#     catpair=c
#     catpair
#     a=catpair[0]
#     b=catpair[1]
#     s=df.groupby([a,b])['ip'].value_counts(normalize=True).rename(f'proba_{a}or{b}_given_ip').reset_index()
#     # s=s.drop(index=[a,b])
#     display(s)
#     input('Next')
#     clear_output()
#     sleep(.5)
    


df.select_dtypes(exclude='number').nunique().sort_values(ascending=False)